In [1]:
import sys
sys.path.append('./Modeling')
sys.path.append('./Data')

In [3]:
# !pip install pyspark

     -------------------------------------- 310.8/310.8 MB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 200.5/200.5 kB 6.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285411 sha256=27c84137f474e3065c981f656f209ae346e871cb6cddf8f4c6c4cebf8d1b2d7c
  Stored in directory: c:\users\eunbi\appdata\local\pip\cache\wheels\2b\9a\39\d8019ffbfb76a39433455e3d5799e94d3e3cae8f41229f6bf8
Successfully built pyspark


In [6]:
from regression import get_parquet_data, log_transformation, encoding_data, split_data, randomforest_grid_search, save_model, load_model

In [6]:
# 전처리 완료 데이터 불러오기
file_path = './Data/csv'

ride_df, alight_df = get_parquet_data(file_path)

TypeError: get_parquet_data() missing 1 required positional argument: 'file_path'

: 

In [5]:
# 데이터프레임 형태 확인
ride_df.info()
alight_df.info()

# 왜도, 첨도 확인

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10, 10))
sns.distplot(ride_df['RIDE'])

plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
sns.distplot(alight_df['ALIGHT'])

plt.show()

# 로그 변환, 스케일링, 인코딩

In [ ]:
# 로그 변환
ride_df = log_transformation(ride_df, RIDE)
alight_df = log_transformation(alight_df, ALIGHT)

In [ ]:
print("Skewness: {:.3f}".format(ride_df['RIDE_log'].skew()))
print("Kurtosis: {:.3f}".format(ride_df['RIDE_log'].kurt()))

print("Skewness: {:.3f}".format(alight_df['ALIGHT_log'].skew()))
print("Kurtosis: {:.3f}".format(alight_df['ALIGHT_log'].kurt()))

In [ ]:
# 스케일러, 인코딩 적용
ride_df_encoding = encoding_data(ride_df)
alight_df_encoding = encoding_data(alight_df)

# 훈련 데이터, 학습 데이터 분리

In [ ]:
ride_X_train, ride_y_train, ride_X_test, ride_y_test = split_data(ride_df_encoding, 'RIDE_log')
alight_X_train, alight_y_train, alight_X_test, alight_y_test = split_data(alight_df_encoding, 'alight_log')

# 최적 파라미터 구하기

In [ ]:
rf_best_model_ride = randomforest_grid_search(ride_X_train, ride_y_train)
rf_best_model_alight = randomforest_grid_search(alight_X_train, alight_y_train)

# 모델 저장

In [ ]:
save_model(rf_best_model_ride)
save_model(rf_best_model_alight)

# 승객 수 예측

In [ ]:
# 모델 불러오기
ride_loaded_model = load_model(rf_best_model_ride)
alight_loaded_model = load_model(rf_best_model_alight)

In [ ]:
# 기후 데이터 불러오기
pred_weather_df = get_weather_data(path, weather.csv)

# 기후 데이터 스케일러, 인코딩 적용
pred_weather_df = encoding_data(pred_weather_df)

# 기후 데이터를 바탕으로 승객 수 예측

In [ ]:
ride_predictions = ride_loaded_model.predict(pred_weather_df)
alight_predictions = alight_loaded_model.predict(pred_weather_df)

In [ ]:
# 로그 역변환
ride_reversed_values = np.exp(ride_predictions)
alight_reversed_values = np.exp(alight_predictions)

print("역변환된 값 : ", ride_reversed_values)
print("역변환된 값 : ", alight_reversed_values)